In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score
import pickle
# from sklearn.model_selection import GridSearchCV,cross_val_score
# from sklearn.linear_model import LinearRegression,SGDRegressor
# from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,ExtraTreesRegressor
# from sklearn.svm import SVR,LinearSVR
# from sklearn.neural_network import MLPRegressor

In [62]:
dataset_name = "new_elim"
model_name = "new_elim_gbr"

In [52]:
# read the dataset and seperate X & Y

df = pd.read_csv(f'{dataset_name}_car_dataset_preprocessed.csv')
Y = df[['price']]
X = df
X.drop(['price'], axis=1, inplace=True)

In [53]:
# down scale Y
scale_million = 1000000
Y = Y/scale_million

In [54]:
X

,prod_year,usage,tpi,city_ahvaz,city_isfahan,city_karaj,city_kermanshah,city_mashhad,city_qom,city_shiraz,...,engine_cond_needfix,engine_cond_replaced,front_chassis_colored,front_chassis_impact,front_chassis_intact,back_chassis_colored,back_chassis_impact,back_chassis_intact,gearbox_auto,gearbox_manual
0,1400,22000,3.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
1,1391,153000,4.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
2,1389,240000,4.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
3,1391,235000,6.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
4,1390,267000,9.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907,1397,68000,12.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
1908,1384,300000,12.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
1909,1396,37000,12.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
1910,1385,375000,12.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1


In [55]:
# split dataset to train & test subsets
x_train,x_test, y_train,y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

In [56]:
def model_mesures(model):
    y_pred = model.predict(x_test)
    print('r2 score: ', r2_score(y_test, y_pred), '\n')
    print('mean_squared_error: ', mean_absolute_error(y_test, y_pred), '\n')

In [57]:
# train the Regression Tree

reg = GradientBoostingRegressor(
    alpha=0.9,
    ccp_alpha=0.0,
    criterion='friedman_mse',
    init=None,
    learning_rate=0.1,
    loss='huber',
    max_depth=6,
    max_features=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_samples_leaf=1,
    min_samples_split=2,
    min_weight_fraction_leaf=0.0,
    n_estimators=212,
    n_iter_no_change=None,
    random_state=123,
    subsample=1.0,
    tol=0.0001,
    validation_fraction=0.1,
    verbose=0,
    warm_start=False)
reg.fit(x_train, y_train.values.ravel())

model_mesures(reg)

r2 score:  0.9485918711543351 

mean_squared_error:  13.624453750649169 



In [64]:
# saving the model
pickle.dump(reg, open(f'models/{model_name}_model.sav', 'wb'))

In [69]:
# deploying the model to sever and reload it
pickle.dump(reg, open('./models/main_model.sav', 'wb'))
!pwd
!/usr/bin/rsync -vv ./models/main_model.sav incv:/home/incv/car-price-predictor/models/.
!curl -v car.vftg.xyz/reload

/home/vafian/Workspace/python/car-price-prediction-using-regression-tree
opening connection using: ssh incv rsync --server -vve.LsfxCIvu . /home/incv/car-price-predictor/models/.  (7 args)
delta-transmission enabled
main_model.sav
total: matches=1001  hash_hits=1001  false_alarms=0 data=0

sent 4,093 bytes  received 6,072 bytes  2,904.29 bytes/sec
total size is 1,000,756  speedup is 98.45
*   Trying 193.107.20.225:80...
* Connected to car.vftg.xyz (193.107.20.225) port 80 (#0)
> GET /reload HTTP/1.1
> Host: car.vftg.xyz
> User-Agent: curl/7.74.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< date: Tue, 03 Jan 2023 12:12:51 GMT
< server: uvicorn
< content-length: 2
< content-type: application/json
< 
* Connection #0 to host car.vftg.xyz left intact
{}